<a href="https://colab.research.google.com/github/kanikakaushik12/AI-ML_Project/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import  tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

from plotly import graph_objects as go
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers  import LSTM, GRU, SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.preprocessing import sequence, text
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.callbacks import EarlyStopping

import nltk
import re

In [2]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU', tpu.master())
except ValueError:
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.iniitialized_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy = tf.distribute.get_strategy()

print("REPLICAS:", strategy.num_replicas_in_sync)



REPLICAS: 1


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_dataset/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/validation.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_dataset/test.csv')


In [4]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)


In [5]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [6]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [7]:
def roc_auc(predictions, target):

  fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
  roc_auc = metrics.auc(fpr, tpr)
  return roc_auc

In [8]:
xtrain,xvalid,ytrain,yvalid= train_test_split(train.comment_text.values, train.toxic.values,stratify=train.toxic.values,random_state=42,test_size=0.2,shuffle=True)

In [9]:
token = text.Tokenizer(num_words=None)
max_len = 1500
token.fit_on_texts(list(xtrain)+list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtrain_pad = sequence.pad_sequences(xtrain_seq,maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq,maxlen=max_len)
word_index = token.word_index


In [10]:
%%time
with strategy.scope():
  model= Sequential()
  model.add(Embedding(len(word_index)+1,300,input_length=max_len))
  model.add(SimpleRNN(100))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13089301 (49.93 MB)
Trainable params: 13089301 (49.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 491 ms, sys: 142 ms, total: 633 ms
Wall time: 905 ms


In [11]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 379s 2s/step - loss: 0.3070 - accuracy: 0.9020
Epoch 2/5
150/150 [==============================] - 335s 2s/step - loss: 0.1784 - accuracy: 0.9318
Epoch 3/5
150/150 [==============================] - 340s 2s/step - loss: 0.0415 - accuracy: 0.9872
Epoch 4/5
150/150 [==============================] - 337s 2s/step - loss: 0.0060 - accuracy: 0.9993
Epoch 5/5
150/150 [==============================] - 341s 2s/step - loss: 0.0018 - accuracy: 1.0000


In [12]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 16s 204ms/step
Auc: 0.84%


In [13]:
scores_model =[]
scores_model.append({'Model': 'SimpleRNN','AUC_Score':roc_auc(scores,yvalid)})

In [14]:
xtrain_seq[:1]

[[664,
  65,
  7,
  19,
  2262,
  14102,
  5,
  2262,
  20439,
  6071,
  4,
  71,
  32,
  20440,
  6620,
  39,
  6,
  664,
  65,
  11,
  8,
  20441,
  1502,
  38,
  6072]]

In [15]:
import numpy as np
from tqdm import tqdm

# Correct file path
glove_file_path = r'C:\glove_file.csv'

# Initialize the embeddings index dictionary
embeddings_index = {}

# Open the GloVe file in read mode with UTF-8 encoding
try:
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Reading GloVe file"):
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Found %s word vectors.' % len(embeddings_index))
except FileNotFoundError:
    print("File not found. Please check the path.")



File not found. Please check the path.


In [16]:
embedding_matrix = np.zeros((len(word_index)+1,300))
for word,i in tqdm(word_index.items()):
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 906926.13it/s]


In [17]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13209601 (50.39 MB)
Trainable params: 160501 (626.96 KB)
Non-trainable params: 13049100 (49.78 MB)
_________________________________________________________________
CPU times: user 477 ms, sys: 297 ms, total: 773 ms
Wall time: 764 ms


In [18]:
model.fit(xtrain_pad,ytrain,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 1049s 7s/step - loss: 0.3411 - accuracy: 0.9053
Epoch 2/5
150/150 [==============================] - 1030s 7s/step - loss: 0.3146 - accuracy: 0.9053
Epoch 3/5
150/150 [==============================] - 1024s 7s/step - loss: 0.3145 - accuracy: 0.9053
Epoch 4/5
150/150 [==============================] - 1026s 7s/step - loss: 0.3139 - accuracy: 0.9053
Epoch 5/5
150/150 [==============================] - 1036s 7s/step - loss: 0.3148 - accuracy: 0.9053


In [19]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 54s 705ms/step
AUC: 0.48%


In [ ]:
scores_model.append({'Model': 'LSTM','AUC_Score':roc_auc(scores,yvalid)})

In [21]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(SpatialDropout1D(0.3))
  model.add(GRU(300))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 spatial_dropout1d (Spatial  (None, 1500, 300)         0         
 Dropout1D)                                                      
                                                                 
 gru (GRU)                   (None, 300)               541800    
                                                                 
 dense_2 (Dense)             (None, 1)                 301       
                                                                 
Total params: 13591201 (51.85 MB)
Trainable params: 542101 (2.07 MB)
Non-trainable params: 13049100 (49.78 MB)
_________________________________________________________________
CPU times: user 559 ms, sys: 255 ms, total: 814 ms
Wall time: 690 ms


In [25]:
#model.fit(X_train, y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

model.fit(xtrain_pad,ytrain,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

In [23]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 162s 2s/step
AUC: 0.50%


In [26]:
scores_model.append({'Model':'GRU','AUC_Score':roc_auc(scores,yvalid)})

In [27]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.8368990350518138},
 {'Model': 'LSTM', 'AUC_Score': 0.4822998269496533},
 {'Model': 'GRU', 'AUC_Score': 0.5}]

In [32]:

%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
  model.add(Bidirectional(LSTM(300, dropout =0.3 ,recurrent_dropout=0.3)))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 bidirectional_1 (Bidirecti  (None, 600)               1442400   
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 1)                 601       
                                                                 
Total params: 14492101 (55.28 MB)
Trainable params: 1443001 (5.50 MB)
Non-trainable params: 13049100 (49.78 MB)
_________________________________________________________________
CPU times: user 822 ms, sys: 285 ms, total: 1.11 s
Wall time: 1.14 s


In [ ]:

model.fit(xtrain_pad,ytrain,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

In [34]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

76/76 [==============================] - 616s 8s/step
AUC: 0.50%


In [9]:
scores_model.append({'Model':'Bidirectional LSTM','AUC_Score':roc_auc(scores,yvalid)})

In [10]:
import pandas as pd
result = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Purples')

In [ ]:
fig = go.Figure(go.Funnelarea(text =results.Model,values =result.AUC_Score,title={"position":"top center","text":"Funnel-Chart of sentiment Distribution"}))

fig.show()

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [ ]:
import pandas as pd

train1 = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/nlp_dataset/jigsaw-toxic-comment-train.csv')
valid = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/validation.csv')
test = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/nlp_dataset/test.csv')
sub = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/nlp_dataset/sample_submission.csv')

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    return np.array(all_ids)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192


In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

In [ ]:
x_x_train = fast_enode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_train = train1.toxic.values
y_valid = valid.toxic.values

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)

    model = Model(inputs=input_word_ids ,outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)